In [624]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from decision import haversine
import matplotlib.cm as cm
import seaborn as sns
from datetime import datetime, timedelta, date
from sklearn import preprocessing
import pickle
import random

from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xg

%matplotlib inline

In [2]:
data = pd.read_csv('1_250_full_data.csv',
                        index_col=0,
                        #usecols = ['POLYLINE', 'Destination'],
                        converters={'POLYLINE': lambda x: json.loads(x), 'Destination': lambda x: json.loads(x)})

In [695]:
def traj_score(targets, predicted):
    return np.mean(np.power([haversine(targets[i], predicted[i]) for i in xrange(len(targets))], 2))

def data_processing_to_train(data, get_dummies=True, y_in_list=True):
    #this data also can used for clustering (before creat dummies)
    
    # make y for predictions
    if y_in_list == True:
        y = list(np.array(data['Destination']))
    else:
        y = np.array(data['Destination'])
    
    # add start_lat and start_lon to the dataset
    traj = np.array(data['POLYLINE'])
    data['Start_lon'] = [i[0][0] for i in traj]
    data['Start_lat'] = [i[0][1] for i in traj]
    
    # fill NA values as specific class
    data = data.fillna('0')
    
    # make some columns categorical (str)
    cat_columns = ['CALL_TYPE','DAY_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID']
    data[cat_columns] = data[cat_columns].astype(str)
    
    # drop irrelevant columns
    data = data.drop(['Destination','MISSING_DATA', 'TRIP_ID', 'POLYLINE', 'DAY_TYPE'], 1)
    
    # next step is to work with timestamps
    
    # get list of holidays in Portugal
    pd_holidays = pd.read_csv('Portugal_holidays.csv')
    holidays = [date(pd_holidays.Year[i], pd_holidays.Month[i], pd_holidays.Day[i]) for i in range(len(pd_holidays))]
    del pd_holidays
    
    # get year, month, day, seconds from midnight, weeknumber, weekday

    dates = []
    for tmstmp in data['TIMESTAMP']:
        t = datetime.utcfromtimestamp(tmstmp)
        date_info = []
        date_info.append(t.year) # year
        date_info.append(t.month) # month
        date_info.append(t.isocalendar()[1]) # weeknumber
        date_info.append(t.weekday()) # weekday where Monday is 0 and Sunday is 6
        date_info.append((t.hour * 3600) + (t.minute * 60) + t.second + (t.microsecond / 1000000.0)) # seconds from midnight

        t_date = t.date()
        if t_date in holidays:
            date_info.append('B')
        elif (date_info[3] == 6) or (date_info[3] == 5):
            date_info.append('B')
        elif (t_date + timedelta(1)) in holidays:
            date_info.append('C')
        elif date_info[3] == 4:
            date_info.append('C')
        else:
            date_info.append('A')

        dates.append(date_info)

    dates = np.array(dates)
    dates_columns = ['YEAR', 'MONTH', 'WEEKNUM', 'WEEKDAY', 'SECFROMMID', 'DAY_TYPE']
    dates_df = pd.DataFrame(data = dates, columns = dates_columns)

    data = pd.concat([data, dates_df], axis=1)
    data['SECFROMMID'] = data['SECFROMMID'].astype(float)
    
    if get_dummies == True:
        dummy_cols = ['CALL_TYPE','DAY_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 
              'YEAR', 'MONTH', 'WEEKNUM', 'WEEKDAY']

        data = pd.get_dummies(data, columns=dummy_cols)

        drop_columns = ['CALL_TYPE_A', 'DAY_TYPE_A', 'ORIGIN_CALL_2002.0', 'ORIGIN_STAND_1.0', 'TAXI_ID_20000066.0',
                        'YEAR_2013', 'MONTH_1', 'WEEKNUM_1', 'WEEKDAY_0']
        data = data.drop(drop_columns,1)
        
        return data, y
    else:
        return data, y
         

In [696]:
X, y = data_processing_to_train(data)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state = 0)

### Only best models will be used

In [6]:
min_samples_leaf = len(X_train)/32

# kNN
kNN_regr = MultiOutputRegressor(KNeighborsRegressor(59))
kNN_regr.fit(X_train, y_train)
print traj_score(y_test, kNN_regr.predict(X_test))

# CART
CART_regr = MultiOutputRegressor(DecisionTreeRegressor(max_depth=3,
                                                       random_state=0,
                                                       min_samples_leaf=min_samples_leaf))
CART_regr.fit(X_train, y_train)
print traj_score(y_test, CART_regr.predict(X_test))

# RF
RF_regr = MultiOutputRegressor(RandomForestRegressor(n_estimators=100,
                                                     max_depth=3,
                                                     random_state=0,
                                                     min_samples_leaf=min_samples_leaf))
RF_regr.fit(X_train, y_train)
print traj_score(y_test, RF_regr.predict(X_test))

# AdaBoost
Ada_regr = MultiOutputRegressor(AdaBoostRegressor(n_estimators=10, 
                                                 base_estimator=(DecisionTreeRegressor(max_depth=3)),
                                                 random_state=0,
                                                 learning_rate = 0.1))
Ada_regr.fit(X_train, y_train)
print traj_score(y_test, Ada_regr.predict(X_test))

# XgBoost
XG_regr = MultiOutputRegressor(xg.XGBRegressor(n_estimators=1000,
                                               max_depth=1,
                                               learning_rate=0.1,
                                               seed=0))
XG_regr.fit(X_train, y_train)
print traj_score(y_test, XG_regr.predict(X_test))

19.6914939679
18.2345812708
18.1697651787
18.6459230394
18.7016556094


In [8]:
def stacking(models, train):
    predictions = []
    for model in models:
        predictions.append(model.predict(train))
        
    return predictions

In [579]:
trajectories = list(data['POLYLINE'])
trunc_trajectories = [traj[:random.randint(1, len(traj))] for traj in trajectories]

In [543]:
forest = pickle.load( open( "forest_250_data_10_trees.p", "rb" ) )
tree = pickle.load( open( "tree_250_data.p", "rb" ) )

In [590]:
models = [kNN_regr, CART_regr, RF_regr, Ada_regr, XG_regr]
stack_X = stacking(models, X)

In [591]:
stack_X.append(np.array(forest.predict(trunc_trajectories)))
stack_X.append(np.array(tree.predict(trunc_trajectories)))

In [592]:
len(stack_X)

7

In [597]:
norm_stack_X = (np.array(stack_X[:6]))
#print norm_stack_X.shape
norm_stack_X = np.transpose(norm_stack_X, axes=(1,0,2))

X_train, X_test, y_train, y_test = train_test_split(norm_stack_X, y, test_size=0.30, random_state = 0)

In [569]:
norm_stack_X[1]

array([[ -8.61899141,  41.16067246],
       [ -8.61672136,  41.15295752],
       [ -8.61749112,  41.1576789 ],
       [ -8.61151699,  41.16065157],
       [ -8.61255169,  41.16029739],
       [ -8.62074079,  41.16623177]])

In [424]:
# Make a prediction with coefficients
def sgd_predict_one(coef, row):
    yhat = coef[0]
    for i in range(len(row)):
        yhat = yhat + coef[i + 1] * row[i]
    return yhat
 
# Estimate linear regression coefficients using stochastic gradient descent
def coefficients_sgd(X_train, y_train, l_rate=0.01, n_epoch=5000):
    #coef = [1./(len(X_train[0]))]*len(X_train[0])
    coef = [np.array([1./(len(X_train[0])), 1./(len(X_train[0]))]) for i in range(len(X_train[0]) + 1)]
    print coef
    for epoch in range(n_epoch):
        for i in range(len(X_train)):
            yhat = sgd_predict_one(coef, X_train[i])
            #error = haversine(y_train[i], yhat)
            error = y_train[i] - yhat
            #print 'error', error
            #print 'test', coef
            coef[0] = coef[0] - l_rate * error
            #print 'coef[0]', coef[0]
            for j in range(len(coef) - 1):
                #coef[j + 1] = coef[j + 1] - l_rate * X_train[i][j] * error
                coef[j + 1] = coef[j + 1] - l_rate * X_train[i][j] * error
            #print coef
    return coef    

# Linear Regression Algorithm With Stochastic Gradient Descent
def sgd_predict(X_test, coef):
    return [sgd_predict_one(coef, row) for row in X_test]


In [425]:
#del sgd_coef
sgd_coef = coefficients_sgd(X_train[:10], y_train[:10], n_epoch=1, l_rate=0.0001)
print sgd_coef

[array([ 0.2,  0.2]), array([ 0.2,  0.2]), array([ 0.2,  0.2]), array([ 0.2,  0.2]), array([ 0.2,  0.2]), array([ 0.2,  0.2])]
[array([ 0.200257  ,  0.21078196]), array([ 0.19778463,  0.64379377]), array([ 0.19778423,  0.64379256]), array([ 0.19778425,  0.64382019]), array([ 0.1977847 ,  0.64380248]), array([ 0.19778424,  0.64376752])]


In [430]:
print traj_score(y_test, sgd_predict(X_test, sgd_coef))

5.49816307126e+37


In [598]:
stacked_lons_train = np.transpose(np.array(X_train), axes=(2,0,1))[0]
stacked_lats_train = np.transpose(np.array(X_train), axes=(2,0,1))[1]

lons_train = np.array(y_train)[:,0]
lats_train = np.array(y_train)[:,1]

stacked_lons_test = np.transpose(np.array(X_test), axes=(2,0,1))[0]
stacked_lats_test = np.transpose(np.array(X_test), axes=(2,0,1))[1]

lons_SGD = LinearRegression().fit(stacked_lons_train, lons_train)
lats_SGD = LinearRegression().fit(stacked_lats_train, lats_train)

lons_pred = lons_SGD.predict(stacked_lons_test)
lats_pred = lats_SGD.predict(stacked_lats_test)

predictions = np.column_stack((lons_pred, lats_pred))

print traj_score(y_test, predictions)


19.6511339747


In [663]:
y_lats.itemset()

array([-8.640639, -8.615799, -8.630658, ..., -8.574417, -8.64297 ,
       -8.578422])

In [690]:
stack_col_names = ['kNN_regr', 'CART_regr', 'RF_regr', 'Ada_regr', 'XG_regr', 'Traj_RF', 'Traj_tree']

stacked_lons = np.transpose(np.array(stack_X), axes=(2,0,1))[0]
stacked_lats = np.transpose(np.array(stack_X), axes=(2,0,1))[1]

stacked_lons = pd.DataFrame(data = stacked_lons.T, columns=stack_col_names)
stacked_lats = pd.DataFrame(data = stacked_lats.T, columns=stack_col_names)

stacked_lons = pd.concat([X, stacked_lons], axis=1)
stacked_lats = pd.concat([X, stacked_lats], axis=1)

y_lons = np.array(y)[:,0]
y_lats = np.array(y)[:,1]

X_lons_train, X_lons_test, y_lons_train, y_lons_test = train_test_split(stacked_lons, y_lons, test_size=0.30, random_state = 0)

X_lats_train = stacked_lats.ix[X_lons_train.index]
X_lats_test = stacked_lats.ix[X_lons_test.index]
y_lats_train = np.array([y_lats[idx] for idx in X_lons_train.index])
y_lats_test = np.array([y_lats[idx] for idx in X_lons_test.index])

lons_LM = LinearRegression().fit(X_lons_train.drop('Traj_tree', 1), y_lons_train)
lats_LM = LinearRegression().fit(X_lats_train.drop('Traj_tree', 1), y_lats_train)

In [691]:
print lons_LM.score(X_lons_test.drop('Traj_tree', 1), y_lons_test)
print lats_LM.score(X_lats_test.drop('Traj_tree', 1), y_lats_test)

-0.131444658026
-0.0970478037348


In [692]:
test = np.column_stack((y_lons_test, y_lats_test))

In [693]:
y_lons_test

array([-8.598708, -8.578179, -8.616303, ..., -8.615718, -8.620587,
       -8.596188])

In [694]:
lons_hat = lons_LM.predict(X_lons_test.drop('Traj_tree', 1))
lats_hat = lats_LM.predict(X_lats_test.drop('Traj_tree', 1))

predictions = np.column_stack((lons_hat, lats_hat))

print traj_score(y_test, predictions)

21.4635370564


In [697]:
lm_test = MultiOutputRegressor(LinearRegression()).fit(X_train, y_train)
print traj_score(y_test, kNN_regr.predict(X_test))

19.6914939679


### Try to upload to Kaggle

In [599]:
test_trajs = pd.read_csv('test.csv',
                        #index_col=,
                        #usecols = ['TRIP_ID', 'POLYLINE'],
                        converters={'POLYLINE': lambda x: json.loads(x)})

# add start_lat and start_lon to the dataset
traj = np.array(test_trajs['POLYLINE'])
test_trajs['Start_lon'] = [i[0][0] for i in traj]
test_trajs['Start_lat'] = [i[0][1] for i in traj]

# fill NA values as specific class
test_trajs = test_trajs.fillna('0')

# make some columns categorical (str)
cat_columns = ['CALL_TYPE','DAY_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID']
test_trajs[cat_columns] = test_trajs[cat_columns].astype(str)

# crate data for clusterization
submission = test_trajs['TRIP_ID']
test_trajs = test_trajs.drop(['MISSING_DATA', 'TRIP_ID', 'POLYLINE', 'DAY_TYPE'], 1)

# Work with timestamps

# get list of holidays in Portugal
pd_holidays = pd.read_csv('Portugal_holidays.csv')
holidays = [date(pd_holidays.Year[i], pd_holidays.Month[i], pd_holidays.Day[i]) for i in range(len(pd_holidays))]
del pd_holidays

# what i want to get?
# year, month, day, seconds from midnight, weeknumber, weekday

dates = []
for tmstmp in test_trajs['TIMESTAMP']:
    t = datetime.utcfromtimestamp(tmstmp)
    date_info = []
    date_info.append(t.year) # year
    date_info.append(t.month) # month
    date_info.append(t.isocalendar()[1]) # weeknumber
    date_info.append(t.weekday()) # weekday where Monday is 0 and Sunday is 6
    date_info.append((t.hour * 3600) + (t.minute * 60) + t.second + (t.microsecond / 1000000.0)) # seconds from midnight
    
    t_date = t.date()
    if t_date in holidays:
        date_info.append('B')
    elif (date_info[3] == 6) or (date_info[3] == 5):
        date_info.append('B')
    elif (t_date + timedelta(1)) in holidays:
        date_info.append('C')
    elif date_info[3] == 4:
        date_info.append('C')
    else:
        date_info.append('A')
    
    dates.append(date_info)

dates = np.array(dates)
dates_columns = ['YEAR', 'MONTH', 'WEEKNUM', 'WEEKDAY', 'SECFROMMID', 'DAY_TYPE']
dates_df = pd.DataFrame(data = dates, columns = dates_columns)

test_trajs = pd.concat([test_trajs, dates_df], axis=1)
test_trajs['SECFROMMID'] = test_trajs['SECFROMMID'].astype(float)

dummy_cols = ['CALL_TYPE','DAY_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID', 
              'MONTH', 'WEEKNUM', 'WEEKDAY']

test_trajs = pd.get_dummies(test_trajs, columns=dummy_cols)

drop_columns = ['CALL_TYPE_A', 'DAY_TYPE_A', 'ORIGIN_CALL_2002.0', 'ORIGIN_STAND_1.0', 'TAXI_ID_20000542',
                'MONTH_8', 'WEEKNUM_33', 'WEEKDAY_0']
test_trajs = test_trajs.drop(drop_columns,1)

In [600]:
zero_data = np.zeros((len(test_trajs), len(X.columns)))
df_test = pd.DataFrame(zero_data, columns = X.columns)

for col_name in test_trajs.columns:
    if col_name in df_test.columns:
        df_test[col_name] = test_trajs[col_name]

In [601]:
stack_test = stacking(models, df_test)
stack_test.append(forest.predict(traj))
stack_test.append(tree.predict(traj))

In [613]:
norm_stack_test = (np.array(stack_test[:6]))
norm_stack_test = np.transpose(norm_stack_test, axes=(1,0,2))

stacked_lons_test = np.transpose(np.array(norm_stack_test), axes=(2,0,1))[0]
stacked_lats_test = np.transpose(np.array(norm_stack_test), axes=(2,0,1))[1]

lons_pred = lons_SGD.predict(stacked_lons_test)
lats_pred = lats_SGD.predict(stacked_lats_test)

predictions = np.column_stack((lons_pred, lats_pred))

submission = pd.DataFrame(data = submission)
submission['LATITUDE'] = lats_pred
submission['LONGITUDE'] = lons_pred
submission.to_csv('stacking.csv', index = False)